In [ ]:
! pip install ripser persim

In [ ]:
import ripser, persim, warnings
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from joblib import Parallel, delayed
from functools import partial

## Function that returns bottleneck distance and optionally plots persistence diagram matchings

In [ ]:
def getDist(feature, homologyGroup, matching=False, dgm=False):
  '''
  Takes a 2D array with columns x,y
  and integer for homology group
  TODO: accept 3D time series
  '''
                  # Line has same width and number of X points as feature...          but with y values at zero
  line = np.array([np.linspace(min(feature[:,0]),max(feature[:,0]),len(feature[:,0])),np.zeros_like(feature[:,1])])
  feature_dgm = ripser.ripser(feature)['dgms'][homologyGroup]
  line_dgm = ripser.ripser(line.T)['dgms'][homologyGroup]

  # Ignore warnings for dgm with non-finite death times
  with warnings.catch_warnings():
    warnings.filterwarnings("ignore", message=r"dgm[12] has points with non-finite death times;ignoring those points")
    if dgm:
      persim.bottleneck_matching(feature_dgm, line_dgm, persim.bottleneck( \
        feature_dgm, line_dgm, matching=True)[1], labels=['feature', 'line'])

    if matching:
      return feature_dgm, line_dgm, persim.bottleneck(feature_dgm, line_dgm, matching=True)[1]
    else:
      return persim.bottleneck(feature_dgm, line_dgm)

## Helper functions for bugfixing and parallelization

In [ ]:
def setupBuff(data):
  '''
  Since we have periodic boundary conditions, we need copy the point cloud on
  each end of the domain before looping over the original middle section
  * time, timeRange are not included in this function *
  '''
  if data.ndim==2:
    x,y = data.T
    xRange = x.max() - x.min()
    lbuff, rbuff = data.copy(), data.copy()
    lbuff[:, 0] -= xRange
    rbuff[:, 0] += xRange
    buff = np.vstack((lbuff, data, rbuff))
  elif data.ndim==3:
    # Assumes dims are time,x,y
    x, y = data.reshape(data.shape[1]*data.shape[2],2).T
    xRange = x.max() - x.min()
    buff = np.zeros((data.shape[1]*data.shape[2],2))
    buffx,buffy = buff.T
    for t in range(data.shape[0]):
      lbuff, rbuff = data[t].copy(), data[t].copy()
      lbuff[:, 0] -= xRange
      rbuff[:, 0] += xRange
      buff[t*data.shape[1]:(t+1)*data.shape[1]] = np.vstack((lbuff, data[t], rbuff))
  #buffx,buffy = buff.T
  return x, y, xRange, buff#, buffx, buffy

                                        # 2D is default
def getWindow(pos, offset, buff, time=None, tPos=None, tOffset=None):
  buffx = buff.T[0]
  if data.ndim==2:
    window = buff[(buffx>pos-offset) & (buffx<pos+offset)]
    # Try to prevent 1-cell disconnected components on the edges
    # This is a rare edge case - this mostly doesn't do anything
    # Doesn't seem to matter for homology groups greater than zero
    window[:,1][window[:,0]==window[0,0]] = window[0,1].min()
    window[:,1][window[:,0]==window[-1,0]] = window[-1,1].min()
  elif data.ndim==3:
    raise ValueError("Windows for 3D time series not yet implemented")
    # TODO: figure out how to select 3D window
    window = buff[(buffx>pos-offset) & (buffx<pos+offset)]
    # Try to prevent 1-cell disconnected components on the edges
  return window

def distsAtPos(pos, offsets, homologyGroups, buff):
  '''
  Gets bottleneck distances at an alongshore position given window offsets, homology groups, and buffered data
  '''
  print(pos, flush = True)
  # Each parallel job will return a row of bottleneck distances for all offsets and homology groups
  posDists = np.zeros((len(offsets), len(homologyGroups)))
  for offsetIdx, offset in enumerate(offsets):
    # TODO : 3D case
    window = getWindow(pos, offset, buff)
    for hgIdx,hg in enumerate(homologyGroups):
        posDists[offsetIdx,hgIdx] = getDist(window, hg)
  # Be careful of this indent! Otherwise it will return only the first offset
  return posDists

## Function that loops over data and stores bottleneck distances in a higher order array

In [ ]:
def sliding_windows_par(pointCloud, homologyGroups=None, dx=None, maxWindow=None, minWindow=None, step=None):
  '''
  Accepts 2D snapshots
  TODO: accept 3D time series
  When this is done, each alongshore position has a spectrum of bottleneck
    distances for a range of window sizes centered on those positions
  '''
  x, y, xRange, buff = setupBuff(pointCloud)
  if pointCloud.ndim==3:
    # Create array of timestep values for each x,y row in buff
           # TODO: Use absolute units: time*dt+startTime
    time = np.repeat(range(pointCloud.shape[0]),pointCloud.shape[1])
    timeRange = time.max() - time.min()

  # Prefer input parameters. Defaults are maximum range and resolution (slow).
  if homologyGroups is None:
    homologyGroups = list(range(pointCloud.ndim))
  if dx is None:
    dx = np.diff(x)[np.diff(x)>0].min() # cell size
  # how much to increment between adjacent window sizes
  if step is None:
    step = dx
  # window widths
  if maxWindow is None:
    maxWindow = xRange
  if minWindow is None:
    minWindow = step   # Anything higher creates an arbitrary cutoff

  # I *think* these are correct... but don't sue me
  positions = dx*np.arange(xRange//dx+2)+x.min()
  offsets = step*np.arange(1,maxWindow//step+1)/2
  offsets = offsets[offsets>=minWindow/2]

  print(f"Positions: {positions}")
  print(f"Offsets: {offsets}")
  print(f"Homology Groups: {homologyGroups}")

  # Create a partial function for distsAtPos with fixed arguments
  distsAtPos_partial = partial(distsAtPos, offsets=offsets, homologyGroups=homologyGroups, buff=buff)

  # Execute the inner loop in parallel for each position
  # The results will be a list of arrays, where each array corresponds to a column of 'dists'
  # Parallel() and delayed() each return a function called by Parallel()() and delayed()()
  allPosDists = Parallel(n_jobs=-1, backend='threading')( # -1 means all; backend='threading' necessary for print (at least in colab)
      # delayed() parallelizes the list comprehension (can't just unpack args, expects multiple calls)
      delayed(distsAtPos_partial)(pos) for pos in positions)

  # Reshape the results into the final dists array
  # allPosDists is a list of arrays, each array is (len(offsets), len(homologyGroups))
  # Need (len(homologyGroups), len(offsets), len(positions)))
  # Rearrange dists dims so that homology group is outermost
  dists = np.array(allPosDists).transpose(2,1,0) # Idk why this is different from non-parallel dists

  return dists, positions, offsets*2, homologyGroups

## Ideas for more plots

IF WE HAVE 3 HOMOLOGY GROUPS WE CAN MAKE RGB MOVIES OF THE BOTTLENECK DISTANCES CHANGING OVER TIME!!!

This will only work with a single window size for time at each timestep

For now, try animating plot_heatmap()

Can I increase step and dx to reduce the number of positions and offsets to save computation time?
I don't think so, they are about half the size of the patterns (nyquist frequency), so if step and dx were any larger we could see anomalies but not trends.
Actually, doubling step to 50 (4 sizes) works fine! 3 is fine too but much less impressive

I think the actual dx is 100m. Should probably go back and change in the video processing notebook and the data files. Or is it better to use 0-indexed integers for axes in math?

Are the data files even accurate? dy looks larger than dx. Need to check the ashton paper for dx,dy,dt and size of domain. If these data files are bad, need to regenerate from new model runs :( Probably won't do this for the class project...

Not in this notebook: Make a figure with 8 subplots for the pretty argmax plots stacked over the noPonds scatterplots of the final frame of each video

## Below are functions for precomputing the distance plots across timesteps and animating them
The animation should look like plot_heatmaps in the old notebook

To do:

* Refactor into:
  * "Pre-computation" function with parameters:
    * videoNum
    * noPonds
  * "animation" function with parameters:
    * npz filename
      * videoNum
      * noPonds
    * hspace
    * hg_to_animate
* I wish I could plot just one timestep with this method... not sure if any of the Gemini changes are problematic
  * Gemini uses set_title instead of colorbar label
  * not sure if I can/ how to animate the colorbar label...

* Verify hspace with plot_heatmap() before animating
* Fix order of yMin and yMax outputs, parameters and arguments in the old notebook

In [ ]:
def precompute(videoNum,noPondsStr="_noPonds"):
  '''
  If you wish to animate a different video, you can change `videoNum`
  noPondsStr = "_noPonds" # set to empty to use full point cloud
  '''

  fnBase = f"jgrf217-sup-000{videoNum+1}-ms0{videoNum}{noPondsStr}"
  print(fnBase)
  readin = np.load(f"data/{fnBase}.npz")
  thisCoast = [readin[key] for key in readin.keys()]
  readin.close()

  # Calculate maxY and minY across all timesteps in thisCoast for consistent scatter plot limits
  maxY = max([data_slice[:,1].max() for data_slice in thisCoast])
  minY = min([data_slice[:,1].min() for data_slice in thisCoast])

  # Calculate global min/max x for `thisCoast` to define constant x-axis for animation
  global_min_x = min([data_slice[:,0].min() for data_slice in thisCoast])
  global_max_x = max([data_slice[:,0].max() for data_slice in thisCoast])
  global_x_range = global_max_x - global_min_x

  # Define constant positions and offsets based on the global x-range
  positions = dx_val * np.arange(global_x_range // dx_val + 2) + global_min_x
  offsets = step_val * np.arange(1, maxWindow_val // step_val + 1) / 2
  offsets = offsets[offsets >= minWindow_val / 2]
  sizes = offsets * 2

  # Use the same parameters for sliding_windows_par as in the last execution
  dx_val = 50
  maxWindow_val = 200
  step_val = 50
  minWindow_val = step_val # Default from sliding_windows_par

  # Homology groups are constant (0 and 1 for 2D data)
  homology_groups = [0, 1]

  all_dists_per_timestep = []

  print("Starting pre-computation for animation...")
  for i, data_timestep in enumerate(thisCoast):
      print(f"Processing timestep {i+1}/{len(thisCoast)}", flush=True)

      # Setup buffer for the current timestep's data
      _, _, _, buff_i = setupBuff(data_timestep)

      # Create a partial function for distsAtPos with fixed arguments (buff_i is specific to this timestep)
      distsAtPos_partial = partial(distsAtPos, offsets=offsets, homologyGroups=homology_groups, buff=buff_i)

      # Execute in parallel for each position
      allPosDists_i = Parallel(n_jobs=-1, backend='threading')(
          delayed(distsAtPos_partial)(pos) for pos in positions)

      # Reshape the results into the final dists array
      dists_i = np.array(allPosDists_i).transpose(2,1,0) # (num_homology_groups, num_offsets, num_positions)
      all_dists_per_timestep.append(dists_i)

  print("Pre-computation complete.")

  # Convert list to array for easier indexing
  # Shape: (num_timesteps, num_homology_groups, num_offsets, num_positions)
  all_dists_per_timestep = np.array(all_dists_per_timestep)
  return fnBase, all_dists_per_timestep, positions, sizes, homology_groups, maxY, minY

In [ ]:
for videoNum in range(1,6):
  for noPondsStr in ["", "_noPonds"]:
    fnBase, all_dists, positions, sizes, homologyGroups, maxY, minY = precompute(videoNum, noPondsStr)

    np.savez_compressed(f"{fnBase}_dists.npz", all_dists, positions, sizes, \
                          homologyGroups, maxY, minY)

In [ ]:
def animate(videoNum, homologyGroups=None, noPondsStr="_noPonds", hspace_val=-1):
  '''
  Choose a videoNum and homology group to animate (e.g., 0 for connected components, 1 for loops)
  homologyGroups = [0,1] # Change this to 1 to animate for the other homology group
  noPondsStr = "_noPonds" # set to empty to use full point cloud
  hspace_val = -1 # vertical space between subplots - depends on yMax-yMin and len(sizes)
                  # Verify hspace with plot_heatmap() before animating
  '''

  fnBase = f"jgrf217-sup-000{videoNum+1}-ms0{videoNum}{noPondsStr}"
  print(fnBase)
  readin = np.load(f"data/{fnBase}.npz")
  thisCoast = [readin[key] for key in readin.keys()]
  readin.close()
  readin = np.load(f"data/{fnBase}_dists.npz")
  thisCoast = [readin[key] for key in readin.keys()]
  readin.close()

  # Setup the figure and initial plot
  fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

  # Calculate constant x-extent for the heatmap and scatter plot
  dx = positions[1] - positions[0]
  x_extent = [positions[0] - dx / 2, positions[-1] + dx / 2]

  # Calculate extent for imshow (y-axis is constant, x-axis is now also constant)
  step_y_extent = sizes[1] - sizes[0] if len(sizes) > 1 else sizes[0] # Handle single window size case
  y_extent = [sizes[0] - step_y_extent, sizes[-1]]
  constant_extent = x_extent + y_extent

  anims = []
  if homologyGroups is None:
    homologyGroups = range(2)
  for hg_to_animate in homologyGroups:
    # Initial data for the first frame
    initial_dists = all_dists[0, hg_to_animate]
    initial_data = thisCoast[0]

    im = ax1.imshow(initial_dists, aspect='auto', origin='lower', extent=constant_extent,
                    vmin=0, vmax=all_dists[:, hg_to_animate].max())
    fig.colorbar(im, ax=ax1, orientation='horizontal', location='top', aspect=50) #label=f"Bottleneck Distance for Homology Group {homology_groups[hg_to_animate]}",
    ax1.set_ylabel('Window Size')
    ax1.set_title(f'Homology Group {homologyGroups[hg_to_animate]} - Timestep 1')
    ax1.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off

    sc = ax2.scatter(initial_data[:,0], initial_data[:,1], s=1)
    ax2.set_xlabel('Position')
    ax2.set_ylabel('Y')
    ax2.set_aspect('equal', adjustable='box')
    ax2.set_xlim(*x_extent) # Use the constant x-extent
    scatpad = (maxY - minY) / 20
    ax2.set_ylim([minY - scatpad, maxY + scatpad])

    # hspace_val = -1.1 # As used in printAndPlot
    plt.subplots_adjust(hspace=hspace_val)
    plt.tight_layout()

    # Update function for the animation
    def update(frame):
        # Get pre-computed data for the current frame
        dists_frame = all_dists[frame, hg_to_animate]
        data_frame = thisCoast[frame]

        # Update heatmap
        im.set_data(dists_frame)
        im.set_extent(constant_extent) # Use the constant extent

        # Update scatter plot
        sc.set_offsets(data_frame)
        ax2.set_xlim(*x_extent) # Ensure x-axis of scatter plot matches heatmap (constant)
        ax1.set_title(f'Homology Group {homologyGroups[hg_to_animate]} - Timestep {frame + 1}')

        return [im, sc, ax1.title]

    # Create the animation
    anim = FuncAnimation(fig, update, frames=len(thisCoast), interval=200, blit=True)

    class PillowWriterNG(PillowWriter):
        def finish(self):
                self._frames[0].save(
                    self.outfile, save_all=True, append_images=self._frames[1:],
                    duration=int(1000 / self.fps), loop=None)

    anim.save(f"{fnBase}_H{hg_to_animate}.gif", writer="pillow")#PillowWriterNG())

    anims[hg_to_animate] = anim

  return anims


In [ ]:
%matplotlib inline
plt.rcParams["animation.html"] = "jshtml"

hspace_vals = [, , , , ,]
for hspaceIdx, videoNum in enumerate(range(1,6)):
  for noPondsStr in ["", "_noPonds"]:
    anims = animate(videoNum, noPondsStr=noPondsStr, hspace_val=hspace_vals[hspaceIdx])
    # Display the animation (Colab will render this automatically if %matplotlib inline and jshtml is set)
    anims[0]
    plt.show()
    anims[1]
    plt.show()